In [ ]:
import os
user = os.getenv('USER')
os.chdir(f'/scratch/cd82/{user}/notebooks')

## Linear Regression - Bayesian Multiple Regression using the Stan library 
```Stan``` is a library that implements a Bayesian sampling Markov Chain Monte Carlo algorithm to predict the coeffiecients in a model. It has an advantage that parameters from complex, heirachical models can be estimated.

#### Set up ```cmdstan```
We have a pre-installed version of ```cmdstan``` on the scratch/cd82 filesystem, so we just need to tell ```cmdstanpy``` where it is.

In [ ]:
import numpy as np
import pandas as pd
import json
import cmdstanpy
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.datasets import fetch_california_housing

# Load the dataset
housing = fetch_california_housing()

In [ ]:
import cmdstanpy
import os

# install_dir = '/scratch/cd82/regression_cmdstan' # for NCI installation
install_dir = os.getenv('HOME')

# If we need to install cmdstan
cmdstanpy.install_cmdstan(overwrite=True, dir=install_dir)

# Pass the installation directory to cmdstanpy
cmdstanpy.set_cmdstan_path(install_dir+'/cmdstan-2.36.0/')
print(cmdstanpy.cmdstan_path())

#### Set up our data and visualise


In [ ]:
from sklearn.datasets import make_regression

X = housing.data
y = housing.target

print(X.shape)

plt.figure(figsize=(8, 4))
# Create a box and whisker plot for each feature
X_df = pd.DataFrame(X, columns=[housing.feature_names])
#y_df = pd.DataFrame(y, columns=['y'])

# Create a box and whisker plot for each feature
X_df.boxplot()
plt.title('California House data')
plt.xticks(rotation=45)

plt.ylim(-150, 200)

plt.ylabel('Values')
plt.grid(True)
plt.show()

# Add a constant to the model (intercept)
X_int = sm.add_constant(X)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_int, y, 
    test_size=0.2, 
    random_state=42)


N = X_train.shape[0]
K = X_train.shape[1]
print(X_train.shape)

X_train_np = X_train
print(type(X_train))

##### Output data for cmdstan
The ```cmdstan``` program requires datasets to be saved to a filesystem as ```JSON``` dictionaries.

In [ ]:
# N.B. Convert matrix and vector data to Numpy arrays
# and then add them to the dictionary as lists
stan_data = {'N': N, 'K': K, 'X': X_train_np.tolist(), 'y':y_train.tolist()}

N2 = X_test.shape[0]
K2 = X_test.shape[1]
stan_data_test = {'N': N2, 'K': K2, 'X': X_test.tolist(), 'y':y_test.tolist()}

# install_dir = os.getenv('HOME')
# point to the file on disk
data_file = os.path.join(install_dir, 'stan_data.json')
print("data_file", data_file)

# Save out dataset
with open(data_file, 'w') as file:
    json.dump(stan_data, file, indent=4)


data_file_test = os.path.join(install_dir, 'stan_data_test.json')
print("data_file_test", data_file_test)

# save our test data
with open(data_file_test, 'w') as file:
    json.dump(stan_data_test, file, indent=4)


In [ ]:
from cmdstanpy import CmdStanModel
import time
import os
# Stan model code
stan_model_code = """

// This describes the input data
// Names must match what was saved to JSON data files
data {
    int<lower=0> N;
    int<lower=0> K;
    matrix[N, K] X;
    vector[N] y;
}

// These are what are being modelled
parameters {
    // a constant has been added to the input X data
    // so we do not need to model the intercept seperately
    // real intercept; 
    vector[K] beta;
    real<lower=0> sigma;
    
}

// This is the 'Prior' definition of our model
model {
    beta ~ normal(0,1);          // The prior for our beta terms
    sigma ~ normal(0,1);         // The prior for the error term
    // intercept ~ normal(0, 1); // Not needed due X augmentation with a column of 1's
    y ~ normal(X * beta, sigma);
}
"""

stan_file = os.path.join(install_dir, '/stan_model_code.stan')
with open(stan_file, 'w') as file:
    file.write(stan_model_code)

time.sleep(3)  

print("stan_file:", stan_file)

model = cmdstanpy.CmdStanModel(stan_file=stan_file)

#### Select the model and print info

In [ ]:
print(model)
print(model.exe_info())

In [ ]:
# fit the model
fit2 = model.sample(data=data_file,  iter_sampling=500, chains=4, parallel_chains=2, max_treedepth=15)

In [ ]:
# file_path = '/tmp/tmp0w8evyvi/stan_model_code9dqjz_0u/stan_model_code-20250307123500_0-stdout.txt'
# file_path = '/tmp/tmp0w8evyvi/stan_model_code9dqjz_0u/stan_model_code-20250307123500_1.csv'
# with open(file_path, 'r') as file:
#     content = file.read()
#     print(content)

# mle = model.optimize(data=data_file)
# print(mle.column_names)
# print(mle.optimized_params_dict)

In [ ]:
fit2

In [ ]:
summary = fit2.summary()
print(summary)

In [ ]:
print(fit2.diagnose())

### Compare result with ```statsmodels``` OLS

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

# Fit the linear regression model
model_ols = sm.OLS(y_train,X_train)
results_ols = model_ols.fit()

# Get the R-squared value
r_squared = results_ols.rsquared
print('R sqrd (extracted):', r_squared)

# Print the summary of the model
print(results_ols.summary())

# Make predictions - If we had some other data
y_pred_sm_ols = results_ols.predict(X_test)

#### Generate predictions with a ```stan``` model

In [ ]:
# Generate predictions
# predictions = model.generate_quantities(data=data_file_test, previous_fit=fit2)

beta_samples = fit2.stan_variable('beta')

print(type(fit2))
print('Column names: ', fit2.column_names)
print(type(beta_samples))
print(beta_samples.shape)
betas_best = beta_samples.mean(axis=0)
betas_stdev = beta_samples.std(axis=0)

In [ ]:
def predict(fit: cmdstanpy.stanfit.mcmc.CmdStanMCMC, paramname: str, data: np.ndarray):
    mc_samples = fit.stan_variable(paramname)
    ave_vect = mc_samples.mean(axis=0)
    predictions = np.dot(data,ave_vect)
    return predictions
    

In [ ]:
# Use our beta estimates to predict y from the test data
y_pred_mc = predict(fit2, 'beta', X_test)

In [ ]:
# Evaluate the Stan derived model
mse = mean_squared_error(y_test, y_pred_mc)
print(f"Mean Squared Error: {mse}")
r2 = r2_score(y_test, y_pred_mc)
print(f"R² Score: {r2}")
    

#### Save the samples

In [ ]:
import os
install_dir = os.getenv('HOME')
outputpath =install_dir+'/stan_outputs'
fit2.save_csvfiles(dir=outputpath)

In [ ]:
# Reload using:
import pandas as pd
data = pd.read_csv(data_file)
# Convert the DataFrame to a dictionary
data_dict = data.to_dict(orient='list')
# Compile the Stan model (again if it has been deleted)
stan_file = os.path.join(install_dir, '/stan_model_code.stan')
model = cmdstanpy.CmdStanModel(stan_file=stan_file)
# Fit the model with the loaded data
fit_from_disk = model.sample(data=data_dict)

## Visualisation

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
# Create a box and whisker plot for each feature
beta_stan_df = pd.DataFrame(beta_samples,columns=[['intercept'] + housing.feature_names])

# Create a box and whisker plot for each feature
beta_stan_df.boxplot()
plt.title('Synthetic MLR data')
plt.xticks(rotation=45)

plt.ylim(-26, 2)

plt.ylabel('Values')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
print(beta_samples.shape)
plt.figure(figsize=(4, 3))
plt.hist(beta_samples[:, 4], bins=100, edgecolor='black')
plt.title('Histogram of samples (Ave Bedrooms)')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()